In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Описание проекта**

Индивидуальный
предприниматель , только что открывший кофейню по адресу г. Тверь, ул. Трёхсвятская,
д.3,планирует использовать умный сервис "Маркетолог", для того чтобы привлечь как можно больше новых посетителей из числа абонентов.
  
**Наша задача** - использовать данные, которые у нас имеются, чтобы:

* Описать алгоритм, который позволит ему собрать сегмент абонентов.

* Предложить наиболее подходящий способ привлечения новых клиентов.   

* Описать выгрузку сегмента в виде SQL-запроса.


**Описание датасетов:**


* cnum_d: таблица со звонками всех абонентов МТС по всей России cnum_d с полями:
msisdn (номер абонента), ts (дата и время звонка), called_party_number (номер,
с которого или на который звонил абонент, в случае sms в этом поле может быть
название компании-отправителя или короткий номер), call_type (тип вызова sms -
2 или звонок - 1), direction (направление вызова - входящий (I) или исходящий

* host_d: таблица с данными посещения хостов всех абонентов МТС по всей России
host_id с полями: msisdn (номер абонента), ts (дата и время посещения), host
(URL сайта типа yandex.ru)

* geo_d: таблица с данными, обновляемыми раз в 10 минут о местонахождении абонента
geo_d с полями: msisdn (номер абонента), ts (дата и время определения
местонахождения), geo_area_key (идентификатор зоны местонахождения абонента)


* twogis_phones: справочник 2GIS, в котором есть рубрика, к которой относится
номер телефона организаций и название самой организации, т.е. таблица twogis с
полями: rubric, phone_num, organization_name

* twogis_hosts: справочник 2GIS, в котором есть рубрика, к которой относится сайт
организации и название самой организации, т.е. таблица twogis с полями: rubric,
host_name, organization_name

**Ход исследования:**     

Данные мы получим из файла,предоставленного заказчиком исследования. О качестве  данных ничего не известно. Поэтому перед выполнением исследования понадобится провести обзор данных.


**Исследование пройдет в 3 этапа:**

1.Обзор данных

2.Исследовательский анализ данных

3.Составление портрета абонента, путем составления SQL запроса.

4.Рекомендации по привлечению новых клиентов.

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('cnum_d.csv')

In [4]:
# посмотрим содержание таблицы со звонками и смс абонентов
data.head()

,msisdn,ts,called_party_number,call_type,direction
0,565cf6bccddd71d0e381358f96a5743e,2022-07-19 02:01:05,900,2,I
1,565cf6bccddd71d0e381358f96a5743e,2022-04-09 07:47:58,79181326867,1,O
2,565cf6bccddd71d0e381358f96a5743e,2022-03-06 08:27:19,79892953676,1,I
3,565cf6bccddd71d0e381358f96a5743e,2022-11-18 12:48:22,79181846285,1,I
4,565cf6bccddd71d0e381358f96a5743e,2022-07-17 10:48:34,79180152465,1,I


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   msisdn               100 non-null    object
 1   ts                   100 non-null    object
 2   called_party_number  100 non-null    object
 3   call_type            100 non-null    int64 
 4   direction            100 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [ ]:
data['msisdn'].unique()

array(['565cf6bccddd71d0e381358f96a5743e',
       'dd67bf4a0e9b1bbbb988ee3db15c16bb',
       '71817322d995da2487379343ddab024e',
       'c483f2e7a684c07df6c347123e3e7a03',
       'bd4eb0ae5d0b357e47e0e06b709198d0',
       '7a77d9c58e059acb6eb7debd5ba158f5',
       '76f70931aaba516a2dfc8180256ff00d',
       '23121f145cd96298a074c3a954f4bd52',
       '66b0622396613f6f1b7f3f4e5a509f18',
       '3c87ccaa52fbc0ccce0a97a647d81a2c'], dtype=object)

Всего 10 клиентов


Для того, чтобы помочь привлечь как можно больше новых посетителей, нам прежде всего требуется составить портрет нашего будущего клиента.

Рассмотрим имеющиеся данные:
Сделав первичный осмотр данных, мы получили датасет размером в 100 строк.
Пропусков не обнаружено.
В столбце msisdn таблицы cnum_d есть 10 уникальных значений.

У нас нет более детальных данных о возрасте, поле абонентов, поэтому посмотрим данные о местонахождении абонента и его интересах.
Для этого посмотрим его онлайн-активность, звонковую активность, SMS-активность.
Необходимо рассмотреть список сайтов или телефонов, к которым могла бы проявлять интерес наша целевая аудитория. Используем эти данные, чтобы запускать таргетированные рассылки. Для максимальной эффективности рассылок можно использовать фильтр по геолокации. В охвате останется только аудитория с соответствующим интересом.


In [8]:
# рассмотрим таблицу с данными о посещении хостов всех абонентов
host = pd.read_csv('/content/host_d.csv')

In [9]:
host.head()

,msisdn,ts,host
0,3ae06583aa2560b723ae06583aaf619f,2022-06-18 01:16:12,hwid.platform.dbankcloud.ru
1,3ae06583aa2560b723ae06583aaf619f,2022-02-26 04:02:46,assets.applovin.com
2,3ae06583aa2560b723ae06583aaf619f,2022-08-23 05:21:01,uc-drru.hispace.dbankcloud.ru
3,3ae06583aa2560b723ae06583aaf619f,2022-08-22 06:38:31,masters-masters.ru
4,3ae06583aa2560b723ae06583aaf619f,2022-08-24 05:02:32,gnpfesdk-pa.googleapis.com


In [10]:
host.describe()

,msisdn,ts,host
count,100,100,100
unique,6,88,96
top,2560b723ae06583aaf619fd9fbd0ae90,2022-09-09 10:06:44,masters-masters.ru
freq,72,8,3


In [12]:
# таблица с данными о местонахождении абонентов
geo = pd.read_csv('/content/geo_d.csv')

In [13]:
geo.head()

,msisdn,ts,geo_area_key
0,5fb9cedebecd39fc7886126b2a20d819,2022-02-03 00:53:00,87283601419
1,5fb9cedebecd39fc7886126b2a20d819,2022-07-22 11:02:29,87283601419
2,f1f96cd7405b19396660b1a650e476ba,2022-02-05 09:21:01,87283601419
3,c46b509b519c69946d20fde5b108e5ef,2022-12-12 01:32:07,87283601419
4,2f395ff20634c956f112b98a8e0dffbb,2022-09-22 06:55:39,87283601419


In [14]:
max_date = data['ts'].max()
min_date = data['ts'].min()
print(f'Данные представлены за период c {min_date} по {max_date}')

Данные представлены за период c 2022-01-02 04:40:33 по 2022-12-26 06:01:27


In [16]:
# справочник 2GIS, в котором есть рубрика, к которой относится номер телефона организаций
gisphone = pd.read_csv('/content/twogis_phones.csv')

In [17]:
gisphone.head(5)

,rubric,phone_num,organization_name
0,Памятники / Надгробия,3422021066,"Память, ритуальное агентство"
1,Юридические услуги,3422021070,"Томилов, Трутнев Групп, ООО, юридическая компания"
2,Заказ автобусов,3422021082,"АТЛАНТ ВОЯЖ, служба заказа автобусов"
3,Продажа земельных участков / малоэтажных домов,3422021088,"Болдино Вилладж, компания по продаже земельных..."
4,Доставка готовых блюд,3422021100,"Diesel room, городское кафе"


In [18]:
gisphone['rubric'].unique()

array(['Памятники / Надгробия', 'Юридические услуги', 'Заказ автобусов',
       'Продажа земельных участков / малоэтажных домов',
       'Доставка готовых блюд', 'Цветы',
       'Утилизация отходов / Переработка вторсырья',
       'Металлическая мебель для помещений', 'Секонд-хенд',
       'Дома престарелых', 'Кофейни',
       'Авторемонт и техобслуживание (СТО)',
       'Теплоизоляционные материалы', 'Портьерные ткани / Шторы',
       'Зоотовары', 'Разработка / поддержка / продвижение web-сайтов',
       'Автозапчасти для отечественных автомобилей',
       'Автозапчасти для иномарок', 'Металлоизделия',
       'Отделочные материалы', 'Кровельные материалы',
       'Туристические агентства', 'Спортивные секции',
       'Агентства недвижимости', 'Аренда спецтехники',
       'Организация и проведение праздников', 'Жалюзи',
       'Котельное оборудование / Котлы', 'Бани / Сауны',
       'Междугородные автогрузоперевозки', 'Аренда спортивных площадок',
       'Профессиональная уборка', 'Аре

Нас интересует рубрика "Кофейни".

In [19]:
# справочник 2GIS, в котором есть рубрика, к которой относится сайт организации и название организации
gishosts = pd.read_csv('/content/twogis_hosts.csv')

In [20]:
gishosts['rubric'].unique()

array(['Новостройки', 'Оценка собственности', 'Детская одежда',
       'Безалкогольные напитки', 'Автомобили', 'Лифты', 'Входные двери',
       'Нотариальные услуги', 'Электрические транспортные средства',
       'Обучение по охране труда', 'Строительство дач / коттеджей',
       'Косметика / Парфюмерия', 'Школы-интернаты',
       'Товары для праздничного оформления / организации праздников',
       'Стоматологические центры', 'Справочно-информационные услуги',
       'Аптеки', 'Бытовая техника', 'Системы безопасности и охраны',
       'Страхование', 'Юридические услуги', 'Авиабилеты',
       'Копировальные услуги', 'Агентства недвижимости',
       'Ремонт / отделка помещений', 'Автозапчасти для иномарок',
       'Парикмахерские', 'Сувениры', 'Товары для охоты',
       'Пункты технического осмотра транспорта', 'Дизайн рекламы',
       'Оборудование для салонов красоты',
       'Спецодежда / Средства индивидуальной защиты',
       'Противопожарное оборудование / инвентарь', 'Системы пер

In [21]:
# справочник с идентификаторами зон
cities_region = pd.read_csv('/content/cities_regions.csv')

In [22]:
cities_region.head()

,geo_area_key,city_name,region,city_area
0,86386033426,Москва,Москва,Медведково
1,86125986808,Москва,Москва,Царицыно
2,85685581132,Москва,Москва,Медведково
3,86348284631,Москва,Москва,Измайлово
4,85949821575,Москва,Москва,Бабушкинский


In [23]:
cities_region.value_counts()

geo_area_key  city_name  region             city_area      
85094205643   Рязань     Рязанская область  Железнодорожный    1
87237462831   Тверь      Тверская область   Центральный        1
87229076493   Тверь      Тверская область   Пролетарский       1
87229076251   Тверь      Тверская область   Центральный        1
87224882120   Тверь      Тверская область   Московский         1
                                                              ..
85316503816   Рязань     Рязанская область  Московский         1
85245199117   Рязань     Рязанская область  Советский          1
85211646137   Рязань     Рязанская область  Октябрьский        1
85211645788   Рязань     Рязанская область  Московский         1
87304572367   Тверь      Тверская область   Заволжский         1
Length: 90, dtype: int64

Здесь нам требуется найти тех, кто находится в Центральном районе города Тверь (рядом с нашей кофейней).

In [24]:
tver = cities_region.loc[cities_region['city_name']== 'Тверь']

tver.head()

,geo_area_key,city_name,region,city_area
30,87220686734,Тверь,Тверская область,Пролетарский
31,87283601419,Тверь,Тверская область,Центральный
32,87279407057,Тверь,Тверская область,Московский
33,87245851567,Тверь,Тверская область,Заволжский
34,87275212338,Тверь,Тверская область,Пролетарский


In [25]:
tver.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 30 to 59
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   geo_area_key  30 non-null     int64 
 1   city_name     30 non-null     object
 2   region        30 non-null     object
 3   city_area     30 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.2+ KB


В Твери находится 30 абонентов

**Напишем SQL для выгрузки данных:**

WITH    
traffic AS    
(SELECT DISTINCT c.msisdn    
FROM    
cnum_d AS c    
JOIN    
twogis_phones AS tph ON c.called_party_number = tph.phone_num    
WHERE c.call_type = 1 AND    
c.direction = 'O' AND    
tph.rubric = 'Кофейни'    

UNION    
SELECT DISTINCT    
h.msisdn    
FROM    
host_d AS h    
JOIN    
twogis_hosts AS th    
ON h.host = th.host_name
WHERE th.rubric = 'Кофейни'),    
geo AS    
(SELECT DISTINCT    
g.msisdn    
FROM geo_d AS g    
JOIN    
cities_regions AS cr    
ON g.geo_area_key = cr.geo_area_key    
WHERE    
cr.city_name = 'Тверь' AND    
cr.city_area = 'Центральный')   

SELECT DISTINCT    
t.msisdn    
FROM traffic AS t    
JOIN geo AS g    
ON t.msisdn = g.msisdn   

Таблица traffic содержит уникальные номера абонентов (msisdn), которые были использованы для исходящих звонков на кофейни. Она формируется путем объединения таблицы cnum_d с таблицей twogis_phones по полю called_party_number, при условии, что тип звонка (call_type) равен 1, направление (direction) равно "O" и rubric в таблице twogis_phones равна "Кофейни". Затем, таблица host_d объединяется с таблицей twogis_hosts по полю host, с условием, что rubric в таблице twogis_hosts равна "Кофейни". В результате в таблице traffic будут содержаться только уникальные номера абонентов, связанные с кофейнями.

Таблица geo содержит уникальные номера абонентов (msisdn), связанные с городом Тверь и районом Центральный. Она формируется путем объединения таблицы geo_d с таблицей cities_regions по полю geo_area_key, при условии, что city_name равно "Тверь" и city_area равно "Центральный". В результате в таблице geo будут содержаться только уникальные номера абонентов, связанные с указанным городом и районом.

В основном запросе происходит объединение таблиц traffic и geo по полю msisdn, чтобы получить уникальные номера абонентов, которые связаны и с кофейнями, и с указанным городом и районом. Результатом запроса будет список уникальных номеров телефонов, которые соответствуют заданным условиям.

**Способы привлечения клиентов:**
1. Рассылка sms, например, с таким текстом «Новая кофейня на ул. Трёхсвятская, д.3,низкие цены, широкий ассортимент, первым посетителям скидки!».
2. Обзвон абонентов с рекламой и предложением скидок.